In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = ""

GITHUB_USERNAME = "Codfishz"
REPO_NAME       = "ASR"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'ASR'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 55), reused 75 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 12.29 MiB | 13.32 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
!cd {REPO_NAME} && git pull

Already up to date.


In [4]:
import os
os.chdir('ASR/Script')

In [5]:
%run "Datapreparation_YOLO.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/crop-pest-and-disease-detection
License(s): CC0-1.0
✅ Total images found in dataset: 25220
Images before filter: 25220


Saving train: 100%|██████████| 20176/20176 [00:31<00:00, 631.76it/s]



train split summary:
  Total images: 20093



Saving val: 100%|██████████| 2522/2522 [00:03<00:00, 649.93it/s]



val split summary:
  Total images: 2514



Saving test: 100%|██████████| 2522/2522 [00:03<00:00, 646.65it/s]


test split summary:
  Total images: 2519

  Images after filter : 25126



In [7]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()

Ultralytics 8.3.112 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 43.6/235.7 GB disk)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from ultralytics import YOLO
print("GPU available:", torch.cuda.is_available())

GPU available: True


# Active YOLO functions

In [9]:
def setup_active_dataset():

    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    active_train_dir = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR)
    os.makedirs(active_train_dir, exist_ok=True)

    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            os.makedirs(os.path.join(active_train_dir, cls), exist_ok=True)
    print(f"Complete creating train_dir {active_train_dir}")

    orig_val_dir = os.path.join(ORIGINAL_DATASET_DIR, VAL_SUBDIR)
    active_val_dir = os.path.join(ACTIVE_DATASET_DIR, VAL_SUBDIR)
    if os.path.exists(orig_val_dir):
        if os.path.exists(active_val_dir):
            shutil.rmtree(active_val_dir)
        shutil.copytree(orig_val_dir, active_val_dir)

    orig_test_dir = os.path.join(ORIGINAL_DATASET_DIR, TEST_SUBDIR)
    active_test_dir = os.path.join(ACTIVE_DATASET_DIR, TEST_SUBDIR)
    if os.path.exists(orig_test_dir):
        if os.path.exists(active_test_dir):
            shutil.rmtree(active_test_dir)
        shutil.copytree(orig_test_dir, active_test_dir)

In [10]:
def get_all_samples():

    samples = []
    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            image_files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
            for f in image_files:
                samples.append((cls, f))
    return samples

In [11]:
import shutil
def copy_samples(sample_list):

    for cls, file_name in sample_list:
        src_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        dst_path = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        if not os.path.exists(dst_path):
            shutil.copy(src_path, dst_path)

In [12]:
import random
def stratified_sample(all_samples, n_initial):

    samples_by_class = {}
    for cls, filename in all_samples:
        samples_by_class.setdefault(cls, []).append((cls, filename))

    stratified = []
    for cls, samples in samples_by_class.items():
        stratified.append(random.choice(samples))
    remaining_count = n_initial - len(stratified)
    if remaining_count > 0:
        remaining_samples = list(set(all_samples) - set(stratified))
        additional_samples = random.sample(remaining_samples, remaining_count)
        stratified.extend(additional_samples)

    return stratified



# Hyperparameters

In [13]:
ORIGINAL_DATASET_DIR = '/content/data_yolo'
ACTIVE_DATASET_DIR = '/content/data_active'

TRAIN_SUBDIR = 'train'
VAL_SUBDIR = 'val'
TEST_SUBDIR = 'test'
Num_Train = 20103
N_INITIAL = int(0.2 * Num_Train)
N_PER_PHASE = int(0.01 * Num_Train)
NUM_PHASES = 50

# TTA

In [14]:
from tqdm import tqdm
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T

def compute_entropy(probs: np.ndarray) -> np.ndarray:
    """Compute entropy for an array of probability distributions."""
    return -np.sum(probs * np.log(probs + 1e-12), axis=1)

def select_tta_samples(model,
                       remaining_samples,
                       n_to_add,
                       K=5,
                       batch_size=32,
                       device='cuda'):
    """
    Batch‑accelerated TTA selection with robust unpacking of logits:
      1) 对每种增强方式，分批次对所有候选样本做推理；
      2) 累加各样本的预测概率；
      3) 最后计算平均概率的熵，挑出熵最高的 n_to_add 样本。
    """
    # 1. 预处理 & 增强列表
    transform_base = T.Compose([
        T.Resize((640, 640)),
        T.ToTensor(),
    ])
    aug_transforms = [
        transform_base,
        T.Compose([T.RandomHorizontalFlip(1.0), transform_base]),
        T.Compose([T.RandomVerticalFlip(1.0), transform_base]),
        T.Compose([T.RandomRotation(15), transform_base]),
        T.Compose([T.ColorJitter(0.2,0.2), transform_base]),
    ]
    K = min(K, len(aug_transforms))

    # 2. 初始化累加容器
    n = len(remaining_samples)
    n_classes = len(model.names)
    probs_accum = np.zeros((n, n_classes), dtype=np.float32)

    # 3. 置 eval & to(device)
    model.eval()
    model.model.to(device)

    with torch.no_grad():
        for k in range(K):
            transform = aug_transforms[k]
            desc = f"TTA aug {k+1}/{K}"
            for start in tqdm(range(0, n, batch_size), desc=desc, unit="batch"):
                batch = remaining_samples[start:start+batch_size]
                imgs = []
                for cls, fname in batch:
                    path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, fname)
                    imgs.append(transform(Image.open(path).convert('RGB')))
                batch_tensor = torch.stack(imgs, 0).to(device)

                # 兼容不同 forward 返回格式
                out = model.model(batch_tensor)
                if isinstance(out, (tuple, list)):
                    logits = out[0]
                else:
                    logits = out

                # 计算概率
                probs = torch.softmax(logits, dim=1)  # [B, C]
                probs = probs.cpu().numpy()

                # 累加
                end = start + probs.shape[0]
                probs_accum[start:end] += probs

    # 4. 平均 & 熵 & 选 top
    avg_probs = probs_accum / K        # [n, C]
    ent = compute_entropy(avg_probs)   # [n]
    idxs = np.argsort(-ent)[:n_to_add] # 取熵最大

    # 5. 返回样本
    return [remaining_samples[i] for i in idxs]


In [ ]:
import csv

# Initialize dataset
setup_active_dataset()
all_samples = get_all_samples()

# Initial sampling and train
current_sample_list = stratified_sample(all_samples, N_INITIAL)
copy_samples(current_sample_list)

print("Initial Epoch")
model = YOLO('yolo11n-cls.pt')
results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name="active-phase0", project="runs/classify")

# Log initial accuracy
acc1 = results.top1
acc5 = results.top5


# Compute remaining samples
remaining_samples = list(set(all_samples) - set(current_sample_list))

# Active Learning (Uncertainty Sampling)
for phase in range(1, NUM_PHASES):
    n_to_add = min(N_PER_PHASE, len(remaining_samples))
    if n_to_add <= 0:
        print("No more samples")
        break

    print(f"Phase {phase}: selecting top {n_to_add} uncertain samples...")

    # Load model from previous phase
    model = YOLO(f"runs/classify/active-phase{phase-1}/weights/last.pt")

    # Apply uncertainty sampling using TTA
    # Send to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.model = model.model.to(device)

    new_samples = select_tta_samples(model, remaining_samples, n_to_add, K=5, device=device)
    copy_samples(new_samples)

    current_sample_list.extend(new_samples)
    remaining_samples = list(set(remaining_samples) - set(new_samples))

    print(f"Phase {phase}: add {n_to_add} samples, total sample number: {len(current_sample_list)}")

    # Train model for this phase
    results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name=f"active-phase{phase}", project="runs/classify")

    # Log accuracy
    acc1 = results.top1
    acc5 = results.top5

    print(f"Phase {phase+1} training completed")


Complete creating train_dir /content/data_active/train
Initial Epoch


100%|██████████| 5.52M/5.52M [00:00<00:00, 301MB/s]

Ultralytics 8.3.112 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data_active, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/classify, name=active-phase0, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

  8                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  9                  -1  1    249728  ultralytics.nn.modules.block.C2PSA           [256, 256, 1]                 
 10                  -1  1    358422  ultralytics.nn.modules.head.Classify         [256, 22]                     
YOLO11n-cls summary: 86 layers, 1,559,286 parameters, 1,559,286 gradients, 3.3 GFLOPs
Transferred 234/236 items from pretrained weights
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 380MB/s]


...

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/classify/active-phase49
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:07<00:00, 10.47it/s]


                   all      0.896          1

1 epochs completed in 0.024 hours.
Optimizer stripped from runs/classify/active-phase49/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/active-phase49/weights/best.pt, 3.2MB

Validating runs/classify/active-phase49/weights/best.pt...
Ultralytics 8.3.112 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n-cls summary (fused): 47 layers, 1,554,206 parameters, 0 gradients, 3.2 GFLOPs
train: /content/data_active/train... found 13869 images in 22 classes ✅ 
val: /content/data_active/val... found 2514 images in 22 classes ✅ 
test: /content/data_active/test... found 2519 images in 22 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:06<00:00, 11.56it/s]


                   all      0.896          1
Speed: 0.5ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/active-phase49
Phase 50 training completed


In [ ]:
import pandas as pd
import csv
import os
import shutil

# Save running log
log_path = "/content/drive/MyDrive/Colab Notebooks/ASR/results/accuracy_log_TTA.csv"
log_columns = ["Phase", "Epoch", "Time", "Train Loss", "Val Loss",
               "Top-1 Accuracy", "Top-5 Accuracy", "LR_pg0", "LR_pg1", "LR_pg2"]
with open(log_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Phase", "Num_Images"] + log_columns[1:])

# Traverse phase dic
for phase in range(NUM_PHASES):
    result_csv_path = os.path.join("runs", "classify", f"active-phase{phase}", "results.csv")
    if not os.path.exists(result_csv_path):
        print(f"Warning: {result_csv_path} not found.")
        continue

    df = pd.read_csv(result_csv_path)
    last_row = df.iloc[-1]

    # Count num of samples
    num_images = N_INITIAL + phase * N_PER_PHASE

    with open(log_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            phase,
            num_images,
            last_row["epoch"],
            last_row["time"],
            last_row["train/loss"],
            last_row["val/loss"],
            last_row["metrics/accuracy_top1"],
            last_row["metrics/accuracy_top5"],
            last_row["lr/pg0"],
            last_row["lr/pg1"],
            last_row["lr/pg2"]
        ])

# Save confusion matrix
# Last phase
last_phase = NUM_PHASES - 1

# Path
base_path = f"/content/ASR/Script/runs/classify/active-phase{last_phase}"
confusion_files = ["confusion_matrix.png", "confusion_matrix_normalized.png"]

# Sey Google Drive dictionary
drive_target_dir = "/content/drive/MyDrive/Colab Notebooks/ASR/results"

# Copy
for filename in confusion_files:
    src = os.path.join(base_path, filename)
    dst = os.path.join(drive_target_dir, f"{filename[:-4]}_TTA.png")
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f"Copied: {filename} → {dst}")
    else:
        print(f"File not found: {src}")


Copied: confusion_matrix.png → /content/drive/MyDrive/Colab Notebooks/ASR/results/confusion_matrix_TTA.png
Copied: confusion_matrix_normalized.png → /content/drive/MyDrive/Colab Notebooks/ASR/results/confusion_matrix_normalized_TTA.png
